In [7]:
import math
import numpy as np
from itertools import permutations

# 1
def p1_bruto(r, n):

    c = 0
    perms = permutations(np.arange(1, n+1))

    for perm in perms:
        perm = np.array(perm)

        aux = np.arange(1, n+1) == perm

        if r == np.count_nonzero(aux):
            c += 1

    return c / math.factorial(n)

def no_coincidencias(k):
    """
    Probabilidad de que en k cartas hayan 0 coincidencias
    """
    if k == 1:
        return 0
    if k == 2:
        return 1

    return (k - 1) * (no_coincidencias(k-1) + no_coincidencias(k-2))

def p1(r, n):
    """
    Calcula la probabilidad de que en n cartas hayan r coincidencias.
    """

    if n < r:
        return 0

    # Casos de exito sobre casos totales
    return math.comb(n, r) * no_coincidencias_iter(n-r) * (1/math.factorial(n))


def no_coincidencias_iter(k):
    """
    # es un intento de hacer `no_coincidencias` usando prog dinamica
    """
    nk2 = 0
    nk1 = 1

    assert k >= 0


    if k in [0, 1]:
        return 0
    if k == 2:
        return 1

    n = 3
    nk = 1

    while n <= k:
    
        nk = (n-1) * (nk1 + nk2)
        nk2 = nk1
        nk1 = nk

        n += 1

    return nk



# b)

esp = sum((r * p1(r, 100) for r in range(0, 101)))
var = sum((r-esp)**2 * p1(r, 100) for r in range(0, 101)) 

print(esp)
print(var)

# c)

def sim_exp():
    deck = np.arange(1, 100)
    perm = np.random.permutation(deck)

    aux = perm == deck
    coincidencias = np.count_nonzero(aux)

    exito_ej_ai = np.count_nonzero(aux[0:9]) == 10
    exito_ej_aii = np.count_nonzero(aux[0:9]) == 10 and np.count_nonzero(aux[9:99]) == 0

    return coincidencias, exito_ej_ai, exito_ej_aii

def sim(n):
    exitos = np.array([0]*101)
    count_ai = 0
    count_aii = 0
    for _ in range(0, n):
        c, ai, aii = sim_exp()
        exitos[c] += 1
        count_ai += int(ai)
        count_aii += int(aii)

    esp = sum((k * (exitos[k]/n) for k in range(0, 101)))

    var = sum(((k - esp)**2 * (exitos[k]/n) for k in range(0, 101)))

    print(f"Esperanza {esp}")
    print(f"Varianza {var}")
    print(f"ai {count_ai / n}")
    print(f"aii {count_aii / n}")

sim(1_000_000)


1.0
0.9999999999999998
Esperanza 0.9999060000000001
Varianza 1.0004099911639999
ai 0.0
aii 0.0


In [94]:
# 2
import time
import lib
import importlib
importlib.reload(lib)


    

N = 10000

# sumatoria posta
start = time.time()
print(sum(np.exp(k/N) for k in range(1, N+1)))
print(time.time() - start)

# simulacion
start = time.time()
print(lib.aprox_sum(1, N, lambda k: np.exp(k/N), 100))
print(time.time() - start)

# 100 primeros terminos
start = time.time()
print(sum(np.exp(k/N) for k in range(1, 101)))
print(time.time() - start)

17183.677439823703
0.027777671813964844
17372.664513726828
0.0005166530609130859
100.50669600897406
0.0004265308380126953


In [95]:
# 3

from random import random
import shutil
import importlib

import lib

importlib.reload(lib)

sim_esp_var = lib.sim_esp_var


def sim():

    # Genera un dado
    tirada = lambda : int(random() * 6) + 1

    # Suma los valores de dos dados
    par = lambda : tirada() + tirada()

    count = 0

    # Conjunto de resultados que salieron
    results = set()

    while len(results) < 12 - 2 + 1:
        results.add(par())
        count += 1

    return count

print("Ejercicio 3".center(shutil.get_terminal_size().columns, "-"))

print(f"a) {sim()}")

for i in [100, 1000, 10_000, 100_000]:
    print("b i)")
    esp, var = sim_esp_var(sim, i)

    print(f"{i}")
    print(f"Esperanza {esp}")
    print(f"Varianza {var}")
    print("\n")

----------------------------------Ejercicio 3-----------------------------------
a) 57
b i)
100
Esperanza 66.1
Varianza 980.3700000000008


b i)
1000
Esperanza 60.497
Varianza 1390.8129909999998


b i)
10000
Esperanza 60.88
Varianza 1403.6493999999998


b i)
100000
Esperanza 61.15494
Varianza 1293.8088735963997




In [96]:
def tres_bii(n):
    ge15 = 0
    le9 = 0
    for _ in range(n):
        x = sim()
        ge15 += int(x >= 15)
        le9 += int(x <= 9)
    
    return ge15/n, le9/n

# Sabemos que no es posible que salgan los 11 valores en 9 tiradas, la prob será 0
# Pueden llegar a salir todos los resultados en 11 tiradas, pero es muy poco probable
# Es muy probable que tome al menos 15 tiradas
for i in [100, 1000, 10_000, 100_000]:
    ge15, le9 = tres_bii(i)

    print(i)
    print(f"Minimo 15={ge15}")
    print(f"Máximo 9={le9}")
    print("\n")

100
Minimo 15=0.98
Máximo 9=0.0


1000
Minimo 15=1.0
Máximo 9=0.0


10000
Minimo 15=0.9985
Máximo 9=0.0


100000
Minimo 15=0.99878
Máximo 9=0.0




In [97]:
# 4

import lib
import time

import importlib
importlib.reload(lib)

rechazo = lib.rechazo
discreta_uniforme = lib.discreta_uniforme

x_vals = list(range(1, 11))
px = [0, .11, .14, .09, .08, .12, .10, .09, .07, .11, .09]

def fx(x):
    try:
        return px[x]
    except IndexError:
        print(f"problemita: {x}")
        return 0

py = lambda _ : 1/10

# a)
generadorA = lambda : rechazo(lambda : discreta_uniforme(1, 10), fx, py, x_vals = x_vals)

# b)
generadorB = lambda : lib.trans_inversa_general(fx, x_vals)

# c)
generadorC = lambda : lib.urna(fx, x_vals)

timeA = 0
timeB = 0
timeC = 0

start = time.time()
esp, var = lib.sim_esp_var(generadorA, 10_000)
print(esp, var)
print("Tiempo A: ", time.time() - start)

start = time.time()
esp, var = lib.sim_esp_var(generadorA, 10_000)
print(esp, var)
print("Tiempo B: ", time.time() - start)

start = time.time()
esp, var = lib.sim_esp_var(generadorA, 10_000)
print(esp, var)
print("Tiempo C: ", time.time() - start)



5.271 8.094559
Tiempo A:  0.08658528327941895
5.2448 8.158072959999998
Tiempo B:  0.07863950729370117
5.2743 8.308559510000002
Tiempo C:  0.0723726749420166


In [98]:
# 5

import importlib 
import math
from lib import trans_inversa_general
from random import random
importlib.reload(lib)

def fx(x, n, p):
    if x > n:
        return 0
    if x < 0:
        return 0
    return math.comb(n, x) * (p**x) * (1-p)**(n-x)

get_generador_binomial_cono = lambda n, p: lambda : trans_inversa_general(lambda x : fx(x, n, p), list(range(0, 11)))

def get_generador_binomial_simulador(n, p):

    def generador_binomial_simulador():
        exitos = 0

        for _ in range(n):
            u = random()
            exitos += int(u < p)
        
        return exitos
    
    return generador_binomial_simulador

gen_cono = get_generador_binomial_cono(10, 0.3)
gen_sim = get_generador_binomial_simulador(10, 0.3)

start = time.time()
for _ in range(10_000):
    gen_cono()
print(f"Tiempo Transformada Inversa: {time.time() - start}")

start = time.time()
for _ in range(10_000):
    gen_sim()
print(f"Tiempo Simulando: {time.time() - start}")

esp, var = lib.sim_esp_var(gen_cono, 10_000)
print(esp, var)
esp, var = lib.sim_esp_var(gen_sim, 10_000)
print(esp, var)


Tiempo Transformada Inversa: 0.05316615104675293
Tiempo Simulando: 0.01603555679321289
3.0124 1.8918462399999996
2.9979 2.1168955900000004


In [100]:
import numpy as np
import lib
import importlib
import time

importlib.reload(lib)

rng = np.random.default_rng()

# 6

def p_binomial(x, n, p):
    if x > n:
        return 0
    if x < 0:
        return 0
    return math.comb(n, x) * (p**x) * (1-p)**(n-x)

px = [0.15, 0.20, 0.10, 0.35, 0.20]

p = lambda x : px[x]

x_vals = list(range(0, 5))

g_trans = lambda : lib.trans_inversa_general_optimizada(p, x_vals)
g_rechazo = lambda : lib.rechazo(lambda : rng.binomial(4, 0.45), p, lambda x : p_binomial(x, 4, 0.45), x_vals=x_vals)

start = time.time()
esp, var = lib.sim_esp_var(g_trans, 10_000)
print(esp, var)
print(time.time() - start)
esp, var = lib.sim_esp_var(g_rechazo, 10_000)
print(esp, var)
print(time.time() - start)

2.256 1.8675640000000007
0.1530146598815918
2.2634 1.8141204400000008
0.43639039993286133


In [68]:
# 7

from typing import Iterable
import lib
import importlib
importlib.reload(lib)
import time

mu = 100

px = lambda k : math.exp(-mu) * mu**(k) * (1/math.factorial(k))
n = 1000

def gt2(it: Iterable):
    return filter(lambda x: x > 2, it)

def rec_poisson(lam):
    def rec(prob):
        return prob * (lam / i)


start = time.time()
esp, var = lib.sim_esp_var(lambda : lib.trans_discreta_general_infinita(px), n)
print(esp, var)
print("tiempo no optimizado: ", time.time() - start)

start = time.time()
esp, var = lib.sim_esp_var(lambda : lib.trans_discreta_general_infinita_new(px, mu=mu, rec=rec_poisson(mu)), n)
print(esp, var)
print("tiempo optimizado: ", time.time() - start)


99.929 158.42695900000035
tiempo no optimizado:  0.25482797622680664
99.417 254.5781109999989
tiempo optimizado:  0.2528808116912842


In [13]:
# 8
import lib
import importlib
import math

importlib.reload(lib)

# a)


# Obtener la funcion de masa de probabilidad
def get_px(k, lam):
    def aux(i):
        return lam**i * (1 / math.factorial(i))

    sumita = sum(aux(j) for j in range(0, k+1))

    def px(x):
        return aux(x) / sumita
    
    return px
k = 10
n = 1000
lam = 0.7

px = get_px(k, lam)

# Metodo general de la transformada inversa
gen_trans = lambda : lib.trans_inversa_general_optimizada(px, list(range(0, k+1)))

# Para el metodo de Rechazo, puedo usar Y ~ Poisson(lambda = 0.7) como soporte
# haciendo el cociente de sus funciones de probabilidad P(X) / P(Y) vemos que
# la constante c sera menor a 1
# en particular es la sumita de abajo en la densidad de X

import numpy as np

rng = np.random.default_rng()

def py(y):
    return math.exp(-lam) * lam**y * (1/math.factorial(y))

gen_rechazo = lambda : lib.rechazo(lambda : rng.poisson(lam), px, py, x_vals=list(range(0, k+1)))

# b)

esp, var = lib.sim_esp_var(gen_trans, n)
print(esp, var)
esp, var = lib.sim_esp_var(gen_rechazo, n)
print(esp, var)

gt2 = 0

for _ in range(n):
    gt2 += int(gen_trans() > 2)

print("Probabilidad de X > 2: ", gt2 / n)

# c)

# Lo importante es ver que cuando truncamos Y en a..b, nuestra nueva funcion de probabilidad es
# P(Y=i)/sum(P(Y=j) for j=a..b)
# Por lo que podemos usar la variable original Y como soporte para la nueva variable
# con c = 1/sum(P(Y=j) for j=a..b)

0.715 0.708775
0.713 0.667631
Probabilidad de X > 2:  0.034


In [89]:
import lib
import importlib
from random import random
importlib.reload(lib)

# 9

# a)

def geom(p):
    return lambda i : ((1-p)**(i-1)) * p

def rec_geom(p):
    return lambda prob : (1-p) * prob

def gen_geom(p):
    return int(math.log(1 - random()) / math.log(1-p)) + 1

p = 0.8
px = lambda i : geom(p)(i+1)
rec = rec_geom(p)
n = 1_000

gen_rec = lambda : lib.trans_discreta_general_infinita(px) + 1
gen_const = lambda :  gen_geom(p)

esp, var = lib.sim_esp_var(gen_rec, n)
print(esp, var)
esp, var = lib.sim_esp_var(gen_const, n)
print(esp, var)

1.237 0.34483099999999967
1.235 0.37877499999999964
4.914 17.971604000000003
5.052 19.135296000000004


In [162]:
# 10
import lib 
import importlib
importlib.reload(lib)

simx = lambda : lib.trans_discreta_general_infinita_new(lambda i : (1/2)**(i + 1))
simy = lambda : lib.trans_discreta_general_infinita_new(lambda i : (2/3)**(i + 1) * 1/2)

gen = lambda : lib.composicion(1/2, simx, simy) + 1

def p(i):
    t1 = (1/2)**(i+1)
    t2 = (1/4) * (2/3)**i
    return  t1 + t2

genpiola = lambda : lib.trans_discreta_general_infinita_new(lambda i : p(i + 1)) + 1 

esp, var = lib.sim_esp_var(gen, 10000)
print(esp, var)

esp, var = lib.sim_esp_var(genpiola, 10000)
print(esp, var)



2.4917 4.0867311100000006
2.5007 4.44009951
